## Searching for bouts for a day of alsa recording

In [1]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2022-02-25 11:24:31,823 root         INFO     Running on pakhi


In [2]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.pipeline import searchbout as sb

### Get the file locations for a session (day) of recordings

In [3]:
reload(et)
sess_par = {'bird': 's_b1238_22',
           'sess': '2022-02-07'}


exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='alsa')

raw_folder = exp_struct['folders']['alsa']
bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts_ceciestunepipe')

In [4]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/s_b1238_22',
 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07',
 'alsa': '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07/alsa',
 'kwik': '/scratch/earneodo/s_b1238_22/alsa/kwik/2022-02-07',
 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1238_22/2022-02-07/alsa',
 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-07/alsa',
 'tmp': '/scratch/earneodo/tmp',
 'msort': '/scratch/earneodo/s_b1238_22/alsa/msort/2022-02-07',
 'ksort': '/scratch/earneodo/s_b1238_22/alsa/ksort/2022-02-07/',
 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-07/alsa/'}

### read a session's bouts pandas
- get the location of the file
- load the pandas pickle
- load the search/spectrogram parameters

In [11]:
reload(sb)
hparams, bpd = sb.read_session_bouts(sess_par['bird'], sess_par['sess'], recording_software='alsa', curated=False)

2022-02-25 11:30:09,712 ceciestunepipe.pipeline.searchbout INFO     No spectrograms in here, will compute...


In [12]:
bpd.head(1)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,170010,177070,8160480,8499360,"[3.1099215238058577, 27.148119136857115, 15.81...",4.099033,108.123583,True,/mnt/sphere/speech_bci/raw_data/s_b1238_22/202...,7060,"[[170010, 170050], [170255, 170320], [170840, ...",12,"[1, 11, 50, 61, 167, 263, 299, 316, 326, 585, ...",26,271.538462,"[37, 25, 27, 29, 25, 35, 45, 41, 39, 44, 25, 3...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True


###### if it wasnt saved (which is a bad mistake), read the sampling rate from the first file in the session

## inspect the bouts and curate them

#### visualize one bout

In [15]:
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets

In [16]:
def viz_one_bout(df: pd.Series, sub_sample=1):
    # get the power and the spectrogram
    sxx = df['spectrogram'][:, ::sub_sample]
    x = df['waveform'][::sub_sample]
    
    # the trace
    tr_waveform = go.Scatter(y=x)
    figwidg_waveform = go.FigureWidget(data=[tr_waveform],
                                      layout= {'height': 300,'width':1000})

    # the spectrogram
    fig_spectrogram = px.imshow(sxx, 
                                     labels={}, 
                                     color_continuous_scale='Inferno',
                                    aspect='auto')

    fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
    fig_spectrogram.update_xaxes(showticklabels=False)
    fig_spectrogram.update_yaxes(showticklabels=False)
    
    
    figwidg_spectrogram = go.FigureWidget(fig_spectrogram)
    
    display(widgets.VBox([figwidg_waveform,
              figwidg_spectrogram]))
    

#viz_one_bout(bpd.iloc[1])

## use it in a widget


In [17]:
### Create a counter object (count goes 1:1 to DataFrame index)
from traitlets import CInt, link

class Counter(widgets.DOMWidget):
    value = CInt(0)
    value.tag(sync=True)

In [18]:
class VizBout():
    def __init__(self, hparams, bouts_pd):
        self.bout = None
        self.bouts_pd = bouts_pd
        self.bout_series = None
        self.is_bout = None
        self.is_confusing = None
        
        self.bout_counter = None
        self.bout_id = None
        
        self.buttons = {}
        self.m_pick = None
        
        
        self.fig_waveform = None
        self.fig_spectrogram = None
        
        self.figwidg_waveform = None
        self.figwidg_spectrogram = None
        
        self.fig_width = 2
        self.sub_sample = 10
        
        self.x = None
        self.sxx = None
        self.tr_waveform = None
        
        self.s_f = hparams['sample_rate']
        
        self.init_fig()
        self.init_widget()
        self.show()
        
    def init_fig(self):
        # the trace
        self.tr_waveform = go.Scatter(y=np.zeros(500))
        self.figwidg_waveform = go.FigureWidget(data=[self.tr_waveform],
                                               layout={'width': 1000, 'height':300})
        
        # the spectrogram
        self.fig_spectrogram = px.imshow(np.random.rand(500, 500), 
                                         labels={}, 
                                         color_continuous_scale='Inferno',
                                        aspect='auto')
        
        self.fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
        self.fig_spectrogram.update_xaxes(showticklabels=False)
        self.fig_spectrogram.update_yaxes(showticklabels=False)
        self.figwidg_spectrogram = go.FigureWidget(self.fig_spectrogram)
        
        
    def init_widget(self):
        # declare elements
        # lay them out
        #
        
        self.bout_counter = Counter()
        self.is_bout = widgets.Checkbox(description='is bout')
        self.is_confusing = widgets.Checkbox(description='Not sure or mixed')
        
        self.buttons['Next'] = widgets.Button(description="Next", button_style='info',
                                              icon='plus')   
        self.buttons['Prev'] = widgets.Button(description="Prev", button_style='warning',
                                              icon='minus')
        self.buttons['Check'] = widgets.Button(description="Check", button_style='success', 
                                               icon='check')
        self.buttons['Uncheck'] = widgets.Button(description="Uncheck", button_style='danger',
                                                 icon='wrong')
        
        [b.on_click(self.button_click) for b in self.buttons.values()]
        
        left_box = widgets.VBox([self.buttons['Prev'], self.buttons['Uncheck']])
        right_box = widgets.VBox([self.buttons['Next'], self.buttons['Check']])
        button_box = widgets.HBox([left_box, right_box])

        self.m_pick = widgets.IntSlider(value=0, min=0, max=self.bouts_pd.index.size-1,step=1, 
                                        description="Bout candidate index")
        
        
        control_box = widgets.HBox([button_box,
                                  widgets.VBox([self.is_bout, self.is_confusing]),
                                    self.m_pick])
        
        link((self.m_pick, 'value'), (self.bout_counter, 'value'))

        self.update_bout()
        
        self.is_bout.observe(self.bout_checked, names='value')
        self.is_confusing.observe(self.confusing_checked, names='value')
        self.m_pick.observe(self.slider_change, names='value')
        
        all_containers = widgets.VBox([control_box, 
                                       self.figwidg_waveform, self.figwidg_spectrogram])
        display(all_containers)
#         display(button_box)
#         display(self.m_pick)
#         display(self.is_bout)
#         display(self.fig)
        
    def button_click(self, button):        
        self.bout_id = self.bout_counter.value
        curr_bout = self.bout_counter
        
        if button.description == 'Next':
            curr_bout.value += 1
        elif button.description == 'Prev':
            curr_bout.value -= 1
        elif button.description == 'Check':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        elif button.description == 'Uncheck':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = False
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        
        # handle the edges of the counter
        if curr_bout.value > self.m_pick.max:
            curr_bout.value = 0
            
        if curr_bout.value < self.m_pick.min:
            curr_bout.value = self.m_pick.max
    
    def slider_change(self, change):
        #logger.info('slider changed')
        #self.bout_counter = change.new
        #clear_output(True)
        self.update_bout()
        self.show()
            
    def bout_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'bout_check'] = bc['new']
    
    def confusing_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'confusing'] = bc['new']
    
    def update_bout(self):
        self.bout_id = self.bout_counter.value
        self.bout_series = self.bouts_pd.iloc[self.bout_id]
        
        self.is_bout.value = bool(self.bout_series['bout_check'])
        self.is_confusing.value = bool(self.bout_series['confusing'])
        
        self.x = self.bout_series['waveform'][::self.sub_sample]
        self.sxx = self.bout_series['spectrogram'][::self.sub_sample]
        
    def show(self):
        #self.fig.clf()
        #self.init_fig()
        # update
#         self.update_bout()
        #plot
        #logger.info('showing')
        
        # Show the figures
        with self.figwidg_waveform.batch_update():
            self.figwidg_waveform.data[0].y = self.x
            self.figwidg_waveform.data[0].x = np.arange(self.x.size) * self.sub_sample / self.s_f 
            
        with self.figwidg_spectrogram.batch_update():
            self.figwidg_spectrogram.data[0].z = np.sqrt(self.sxx[::-1])
            
        

viz_bout = VizBout(hparams, bpd)

In [22]:
np.where(viz_bout.bouts_pd['bout_check']==True)[0].size

54

### save it

In [23]:
### get the curated file path
##save to the curated file path
bouts_curated_file_path = os.path.join(bouts_folder, hparams['bout_curated_file'])

os.makedirs(bouts_folder, exist_ok=True, mode=0o777)

viz_bout.bouts_pd.to_pickle(bouts_curated_file_path)
logger.info('saved curated bout pandas to pickle {}'.format(bouts_curated_file_path))

2022-02-08 11:55:44,428 root         INFO     saved curated bout pandas to pickle /mnt/sphere/speech_bci/derived_data/s_b1238_22/2022-02-07/alsa/bouts_ceciestunepipe/bout_checked.pickle


In [38]:
viz_bout.bouts_pd['file'][0]

'/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-06/alsa/07-00-01-01.wav'

In [24]:
viz_bout.bouts_pd[viz_bout.bouts_pd['bout_check']==True]['file'].values

array(['/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07/alsa/09-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07/alsa/09-00-01-02.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07/alsa/11-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07/alsa/11-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07/alsa/11-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07/alsa/11-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07/alsa/11-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07/alsa/11-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07/alsa/11-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07/alsa/11-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07/alsa/11-00-01-01.wav',
       '/mnt/sphere/speech_bci/raw_data/s_b1238_22/2022-02-07/als

In [87]:
viz_bout.bouts_pd.head(5)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,1772690,1781715,85089120,85522320,"[0.8992322534584924, 9.897367501899893, 9.4748...",2.359551,182.680045,False,/mnt/sphere/speech_bci/raw_data/s_b1238_22/202...,9025,"[[1772690, 1772740], [1772845, 1772910], [1773...",18,"[1, 9, 33, 42, 61, 102, 121, 239, 247, 254, 34...",54,167.129630,"[6, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 6, 5, 4, 3, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
1,1793070,1799925,86067360,86396400,"[6.184213799374133, 5.3566748510823565, 13.362...",2.359551,267.096691,False,/mnt/sphere/speech_bci/raw_data/s_b1238_22/202...,6855,"[[1793070, 1793125], [1793165, 1793350], [1793...",20,"[5, 14, 22, 31, 38, 52, 61, 68, 79, 105, 135, ...",58,118.189655,"[12, 11, 11, 12, 12, 12, 12, 11, 10, 8, 8, 9, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
2,3005,10460,144240,502080,"[0.887261381666812, 1.3398271004562607, 4.4853...",1.675094,120.070598,False,/mnt/sphere/speech_bci/raw_data/s_b1238_22/202...,7455,"[[3005, 3045], [4665, 4765], [5290, 5330], [53...",15,"[3, 334, 343, 355, 371, 396, 407, 458, 478, 49...",59,126.355932,"[-3, -2, -3, -2, -6, -9, -8, -10, -12, -14, -1...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
3,144815,157165,6951120,7543920,"[0.41380735035998034, 0.5234897127529178, 0.58...",1.098511,5.122523,False,/mnt/sphere/speech_bci/raw_data/s_b1238_22/202...,12350,"[[144815, 145765], [146815, 147765], [148815, ...",11,"[3, 12, 33, 40, 47, 54, 61, 72, 82, 89, 100, 1...",94,131.382979,"[11, 10, 10, 7, 6, 5, 3, 2, 2, 3, 2, 3, 4, 3, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
4,1647115,1656980,79061520,79535040,"[1.1034896530762963, 1.261074910541601, 1.2935...",1.641276,124.312637,False,/mnt/sphere/speech_bci/raw_data/s_b1238_22/202...,9865,"[[1647115, 1647150], [1648060, 1648090], [1648...",20,"[3, 122, 132, 150, 172, 194, 289, 344, 358, 36...",91,108.406593,"[-7, -5, -4, -5, -1, -1, -3, -8, -12, -10, -10...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
